In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/MyDrive/Flower Recognition")

In [ ]:
DAISY_SOURCE_DIR = "/content/drive/MyDrive/Flower Recognition/Original Dataset/daisy/"
TRAIN_DAISY_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/TRAIN/daisy/"
TEST_DAISY_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/TEST/daisy/"
VAL_DAISY_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/VAL/daisy/"

DANDELION_SOURCE_DIR = "/content/drive/MyDrive/Flower Recognition/Original Dataset/dandelion/"
TRAIN_DANDELION_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/TRAIN/dandelion/"
TEST_DANDELION_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/TEST/dandelion/"
VAL_DANDELION_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/VAL/dandelion/"

ROSE_SOURCE_DIR = "/content/drive/MyDrive/Flower Recognition/Original Dataset/rose/"
TRAIN_ROSE_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/TRAIN/rose/"
TEST_ROSE_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/TEST/rose/"
VAL_ROSE_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/VAL/rose/"

SUNFLOWER_SOURCE_DIR = "/content/drive/MyDrive/Flower Recognition/Original Dataset/sunflower/"
TRAIN_SUNFLOWER_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/TRAIN/sunflower/"
TEST_SUNFLOWER_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/TEST/sunflower/"
VAL_SUNFLOWER_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/VAL/sunflower/"

TULIP_SOURCE_DIR = "/content/drive/MyDrive/Flower Recognition/Original Dataset/tulip/"
TRAIN_TULIP_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/TRAIN/tulip/"
TEST_TULIP_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/TEST/tulip/"
VAL_TULIP_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/VAL/tulip/"


Checking the number of training images

In [ ]:
print("Number of training daisy images", len(os.listdir(TRAIN_DAISY_DIR)))
print("Number of training dandelion images", len(os.listdir(TRAIN_DANDELION_DIR)))
print("Number of training rose images", len(os.listdir(TRAIN_ROSE_DIR)))
print("Number of training sunflower images", len(os.listdir(TRAIN_SUNFLOWER_DIR)))
print("Number of training tulip images", len(os.listdir(TRAIN_TULIP_DIR)))

Number of training daisy images 692
Number of training dandelion images 949
Number of training rose images 705
Number of training sunflower images 660
Number of training tulip images 885


Checking the number of testing images

In [ ]:
print("Number of testing daisy images", len(os.listdir(TEST_DAISY_DIR)))
print("Number of testing dandelion images", len(os.listdir(TEST_DANDELION_DIR)))
print("Number of testing rose images", len(os.listdir(TEST_ROSE_DIR)))
print("Number of testing sunflower images", len(os.listdir(TEST_SUNFLOWER_DIR)))
print("Number of testing tulip images", len(os.listdir(TEST_TULIP_DIR)))

Number of testing daisy images 23
Number of testing dandelion images 31
Number of testing rose images 23
Number of testing sunflower images 22
Number of testing tulip images 29


Checking the number of validation images

In [ ]:
print("Number of validation daisy images", len(os.listdir(VAL_DAISY_DIR)))
print("Number of validation dandelion images", len(os.listdir(VAL_DANDELION_DIR)))
print("Number of validation rose images", len(os.listdir(VAL_ROSE_DIR)))
print("Number of validation sunflower images", len(os.listdir(VAL_SUNFLOWER_DIR)))
print("Number of validation tulip images", len(os.listdir(VAL_TULIP_DIR)))

Number of validation daisy images 54
Number of validation dandelion images 75
Number of validation rose images 56
Number of validation sunflower images 52
Number of validation tulip images 70


Setting the directories

In [ ]:
TRAINING_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/TRAIN"
# Experiment with your own parameters to reach 99.9% validation accuracy or better
train_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=50,
                                                    class_mode='categorical',
                                                    target_size=(299, 299))

VALIDATION_DIR = "/content/drive/MyDrive/Flower Recognition/Splitted/VAL"
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=50,
                                                              class_mode='categorical',
                                                              target_size=(299, 299))



Found 3889 images belonging to 5 classes.
Found 307 images belonging to 5 classes.


Loading the pre-trained inceptionV3 model

In [ ]:
pre_trained_model = InceptionV3(input_shape=(299, 299, 3),include_top=False,
                                pooling='avg', weights='imagenet',classes=5)


87916544/87910968 [==============================] - 2s 0us/step


In [ ]:
print(pre_trained_model.summary())

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 149, 149, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 149, 149, 32) 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 149, 149, 32) 0           batch_normalization_188[0][0]    
_______________________________________________________________________________________

In [ ]:
# freeze the layers
for layer in pre_trained_model.layers:
    layer.trainable = False

last_layer = pre_trained_model.get_layer('global_average_pooling2d')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output


last layer output shape:  (None, 2048)


In [ ]:
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(last_output)

# Add a final sigmoid layer for classification
x = layers.Dense(5, activation='softmax')(x)

model = Model(pre_trained_model.input, x)
print(model.summary())

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 149, 149, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 149, 149, 32) 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 149, 149, 32) 0           batch_normalization_188[0][0]    
____________________________________________________________________________________________

In [ ]:
# Add a fully connected layer with 1,024 hidden units and ReLU activation
# x = layers.Dense(512, activation='relu')(last_output)
# Add a final sigmoid layer for classification
# x = layers.Dense(5, activation='softmax')(x)

# model = Model(pre_trained_model.input, x)
# print(model.summary())

In [ ]:
# compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

# train the model (adjust the number of epochs from 1 to improve performance)
history = model.fit(
            train_generator,
            validation_data=validation_generator,
            epochs=10,
            verbose=1)

Epoch 1/10
78/78 [==============================] - 87s 1s/step - loss: 1.2148 - acc: 0.6224 - val_loss: 0.5771 - val_acc: 0.7980
Epoch 2/10
78/78 [==============================] - 83s 1s/step - loss: 0.4281 - acc: 0.8490 - val_loss: 0.4100 - val_acc: 0.8632
Epoch 3/10
78/78 [==============================] - 83s 1s/step - loss: 0.3473 - acc: 0.8793 - val_loss: 0.3653 - val_acc: 0.8664
Epoch 4/10
78/78 [==============================] - 83s 1s/step - loss: 0.3701 - acc: 0.8712 - val_loss: 0.3383 - val_acc: 0.8827
Epoch 5/10
78/78 [==============================] - 82s 1s/step - loss: 0.3253 - acc: 0.8868 - val_loss: 0.3163 - val_acc: 0.8925
Epoch 6/10
78/78 [==============================] - 83s 1s/step - loss: 0.3026 - acc: 0.8898 - val_loss: 0.3153 - val_acc: 0.9023
Epoch 7/10
78/78 [==============================] - 82s 1s/step - loss: 0.3064 - acc: 0.8869 - val_loss: 0.4276 - val_acc: 0.8469
Epoch 8/10
78/78 [==============================] - 83s 1s/step - loss: 0.2972 - acc: 0.89

In [ ]:
categories = np.array(['daisy','dandelion','tulip','rose','sunflower'])
categories = np.sort(categories)
print(categories)

['daisy' 'dandelion' 'rose' 'sunflower' 'tulip']


Testing the image

In [ ]:
from keras.preprocessing import image
img = image.load_img('/content/drive/MyDrive/Flower Recognition/Splitted/TEST/daisy/14354051035_1037b30421_n.jpg', 
                     target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

image_tensor = np.vstack([x])
classes = model.predict(image_tensor)
print(classes)
max_arg = np.argmax(classes)
print(categories[max_arg])


[[0.0000000e+00 0.0000000e+00 4.9976269e-29 4.2225756e-06 9.9999583e-01]]
tulip
